In [ ]:
# =============================================================================
# EXPERIMENT: Logistic Regression - Fine Tuned
# Date: 2025-12-XX  
# Description: Going back to LogReg since it performed best (0.78)
# =============================================================================

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
from datetime import datetime
from pathlib import Path

In [2]:
# =============================================================================
# 1. LOAD PREPROCESSED DATA (no need to preprocess again!)
# =============================================================================
print("Loading preprocessed data...")
X = pd.read_pickle('../../data/processed/X_train_processed.pkl')
y = pd.read_pickle('../../data/processed/y_train.pkl')
X_test = pd.read_pickle('../../data/processed/X_test_processed.pkl')
test_ids = pd.read_pickle('../../data/processed/test_ids.pkl')

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Loading preprocessed data...
X shape: (20885, 55)
y shape: (20885,)


In [3]:
# =============================================================================
# 2. DEFINE EXPERIMENT PARAMETERS
# =============================================================================
EXPERIMENT_NAME = "logistic_regression_fine_tuned"
MODEL_DESCRIPTION = "Logistic Regression with exhaustive grid search (beat RF at 0.78)"

In [6]:
# =============================================================================
# 3. HYPERPARAMETER TUNING
# =============================================================================
from sklearn.model_selection import GridSearchCV, StratifiedKFold


print(f"\n{'='*70}")
print(f"EXPERIMENT: {EXPERIMENT_NAME}")
print(f"{'='*70}")
print("\nNote: Your original LogReg got 0.78 - let's improve it!")

# Fine-grained parameter grid
param_grid = {
    'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100],  # More granular
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': ['balanced', None],
    'max_iter': [2000]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\nRunning GridSearchCV...")
print("This may take 5-10 minutes...\n")

grid_search = GridSearchCV(
    LogisticRegression(random_state=42),
    param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X, y)

# =============================================================================
# 4. RESULTS
# =============================================================================
print(f"\n{'='*70}")
print("HYPERPARAMETER TUNING RESULTS")
print(f"{'='*70}")

print(f"\nBest parameters found:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest CV ROC-AUC: {grid_search.best_score_:.4f}")

# Show top 5
results_df = pd.DataFrame(grid_search.cv_results_)
top_5 = results_df.nsmallest(5, 'rank_test_score')[
    ['params', 'mean_test_score', 'std_test_score']
].copy()
print("\nTop 5 parameter combinations:")
for idx, row in top_5.iterrows():
    print(f"  {row['mean_test_score']:.4f} ± {row['std_test_score']:.4f}: {row['params']}")

# =============================================================================
# 5. OVERFITTING ANALYSIS
# =============================================================================
print(f"\n{'='*70}")
print("OVERFITTING ANALYSIS")
print(f"{'='*70}")

model = grid_search.best_estimator_

y_pred_train = model.predict_proba(X)[:, 1]
train_auc = roc_auc_score(y, y_pred_train)
cv_auc = grid_search.best_score_
gap = train_auc - cv_auc

print(f"\nTraining ROC-AUC: {train_auc:.4f}")
print(f"Cross-Validation ROC-AUC: {cv_auc:.4f}")
print(f"Gap (Train - CV): {gap:.4f}")



EXPERIMENT: logistic_regression_fine_tuned

Note: Your original LogReg got 0.78 - let's improve it!

Running GridSearchCV...
This may take 5-10 minutes...

Fitting 5 folds for each of 72 candidates, totalling 360 fits

HYPERPARAMETER TUNING RESULTS

Best parameters found:
  C: 0.05
  class_weight: balanced
  max_iter: 2000
  penalty: l1
  solver: saga

Best CV ROC-AUC: 0.8406

Top 5 parameter combinations:
  0.8406 ± 0.0078: {'C': 0.05, 'class_weight': 'balanced', 'max_iter': 2000, 'penalty': 'l1', 'solver': 'saga'}
  0.8405 ± 0.0080: {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 2000, 'penalty': 'l1', 'solver': 'saga'}
  0.8405 ± 0.0081: {'C': 0.05, 'class_weight': 'balanced', 'max_iter': 2000, 'penalty': 'l1', 'solver': 'liblinear'}
  0.8405 ± 0.0078: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 2000, 'penalty': 'l2', 'solver': 'saga'}
  0.8405 ± 0.0082: {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 2000, 'penalty': 'l1', 'solver': 'liblinear'}

OVERFITTING ANALYSIS

In [8]:
test_proba = model.predict_proba(X_test)[:, 1]


In [9]:
# =============================================================================
# 5. CREATE SUBMISSION FILE
# =============================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
submission_filename = f"../../outputs/predictions/logistic_regression/{EXPERIMENT_NAME}_{timestamp}.csv"

submission = pd.DataFrame({
    'icustay_id': test_ids,
    'prediction': test_proba
})
submission.to_csv(submission_filename, index=False)
print(f"\n✓ Submission saved: {submission_filename}")


✓ Submission saved: ../../outputs/predictions/logistic_regression/logistic_regression_fine_tuned_20251122_0119.csv


In [10]:
# =============================================================================
# 6. LOG EXPERIMENT (CRITICAL FOR TRACKING!)
# =============================================================================
experiment_log = {
    'timestamp': timestamp,
    'experiment_name': EXPERIMENT_NAME,
    'model_type': 'LogisticRegression',
    'cv_roc_auc_mean': cv_auc,
    'train_roc_auc': train_auc,
    'overfitting_gap': gap,
    'kaggle_public_score': None,
    'kaggle_private_score': None,
    'parameters': str(grid_search.best_params_),
    'description': MODEL_DESCRIPTION,
    'submission_file': submission_filename
}

log_file = '../experiment_log.csv'
log_df = pd.DataFrame([experiment_log])

import os
if os.path.exists(log_file):
    log_df.to_csv(log_file, mode='a', header=False, index=False)
else:
    log_df.to_csv(log_file, index=False)

print(f"✓ Experiment logged to {log_file}")

✓ Experiment logged to ../experiment_log.csv


In [11]:
# =============================================================================
# 7. SAVE MODEL (OPTIONAL)
# =============================================================================
import pickle


model_filename = f"../../outputs/models/{EXPERIMENT_NAME}_{timestamp}.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved: {model_filename}")

print(f"\n{'='*70}")
print("EXPERIMENT COMPLETE!")
print(f"{'='*70}")
print(f"\nNext steps:")
print(f"1. Upload {submission_filename} to Kaggle")
print(f"2. Record Kaggle score in {log_file}")
print(f"3. Copy this notebook and modify for next experiment")

✓ Model saved: ../../outputs/models/logistic_regression_fine_tuned_20251122_0119.pkl

EXPERIMENT COMPLETE!

Next steps:
1. Upload ../../outputs/predictions/logistic_regression/logistic_regression_fine_tuned_20251122_0119.csv to Kaggle
2. Record Kaggle score in ../experiment_log.csv
3. Copy this notebook and modify for next experiment
